In [2]:
import keras
from keras.models import Sequential
from keras.layers import Dense,Activation,BatchNormalization
from keras.layers import LeakyReLU,PReLU,ELU
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.activations import relu, sigmoid
from sklearn.model_selection import GridSearchCV



Using Theano backend.
WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
C:\Users\Bhavesh\Miniconda3\lib\site-packages\theano\configdefaults.py:560: UserWarning: DeprecationWarning: there is no c++ compiler.This is deprecated and with Theano 0.11 a c++ compiler will be mandatory
  warnings.warn("DeprecationWarning: there is no c++ compiler."
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV


In [4]:
dataset = pd.read_csv("D:\Data scientist\Dataset\Churn_Modelling.csv")

In [5]:
dataset.head(5)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
dataset.shape

(10000, 14)

In [7]:
X = dataset.iloc[:,3:13]
Y = dataset.iloc[:, 13]


In [8]:
geography = pd.get_dummies(X["Geography"],drop_first=True)
gender = pd.get_dummies(X["Gender"],drop_first=True)

In [9]:
X = pd.concat([X,geography,gender],axis=1)

In [10]:
X = X.drop(['Geography','Gender'],axis=1)

In [11]:
X_train , X_test , Y_train, Y_test = train_test_split(X, Y, random_state = 0 , test_size = 0.2)

In [12]:
sc  = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [13]:
classifier = Sequential()

classifier.add(Dense(units=10 , kernel_initializer=  'he_normal',activation='relu',input_dim= 11))
classifier.add(Dense(units=20 , kernel_initializer=  'he_normal',activation='relu' ))
classifier.add(Dense(units=15 , kernel_initializer=  'he_normal',activation='relu' ))

classifier.add(Dense(units=1 , kernel_initializer=  'glorot_uniform',activation='sigmoid' ))


classifier.compile(optimizer = 'adam' , loss= 'binary_crossentropy',metrics = ['accuracy'])
    
model_history = classifier.fit(X_train,Y_train,validation_split=0.33,batch_size=10,nb_epoch=100)

C:\Users\Bhavesh\Miniconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  if sys.path[0] == '':


Train on 5359 samples, validate on 2641 samples
Epoch 1/100
5359/5359 [==============================] - 56s 10ms/step - loss: 0.4986 - accuracy: 0.7826 - val_loss: 0.4599 - val_accuracy: 0.8054
Epoch 2/100
5359/5359 [==============================] - 52s 10ms/step - loss: 0.4416 - accuracy: 0.8037 - val_loss: 0.4532 - val_accuracy: 0.8111
Epoch 3/100
5359/5359 [==============================] - 54s 10ms/step - loss: 0.4273 - accuracy: 0.8140 - val_loss: 0.4342 - val_accuracy: 0.8092
Epoch 4/100
5359/5359 [==============================] - 51s 9ms/step - loss: 0.4169 - accuracy: 0.8184 - val_loss: 0.4277 - val_accuracy: 0.8152
Epoch 5/100
5359/5359 [==============================] - 49s 9ms/step - loss: 0.4100 - accuracy: 0.8253 - val_loss: 0.4214 - val_accuracy: 0.8171
Epoch 6/100
5359/5359 [==============================] - 49s 9ms/step - loss: 0.4028 - accuracy: 0.8296 - val_loss: 0.4141 - val_accuracy: 0.8228
Epoch 7/100
5359/5359 [==============================] - 48s 9ms/step - l

In [14]:
classifier.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 10)                120       
_________________________________________________________________
dense_2 (Dense)              (None, 20)                220       
_________________________________________________________________
dense_3 (Dense)              (None, 15)                315       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 16        
Total params: 671
Trainable params: 671
Non-trainable params: 0
_________________________________________________________________


In [15]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

cm = confusion_matrix(Y_test, y_pred)


In [16]:
cm


array([[1470,  125],
       [ 178,  227]], dtype=int64)

In [17]:
score =  accuracy_score(y_pred,Y_test)


In [18]:
score


0.8485

In [19]:
def create_model(layers,activation):
    model = Sequential()
    for i, nodes in enumerate(layers):
        if i==0:
            model.add(Dense(nodes,input_dim=X_train.shape[1]))
            model.add(Activation(activation))
            model.add(Dropout(0,3))
            
        else:
            model.add(Dense(nodes))
            model.add(Activation(activation))
            model.add(Dropout(0,3))
            
     
    model.add(Dense(units = 1, kernel_initializer= 'glorot_uniform', activation = 'sigmoid'))
    
    model.compile(optimizer= 'adam', loss= 'binary_crossentropy', metrics= ['accuracy'])
    return model

model = KerasClassifier(build_fn=create_model,verbose=0)
 
    
layers = [[20],  [40,20],[45, 30, 15 ]]
activations = ['sigmoid', 'relu']

param_grid = dict(layers=layers, activation=activations, batch_size = [128,256], epochs=[30])
grid = GridSearchCV(estimator=model, param_grid = param_grid,cv=5)

grid_result = grid.fit(X_train,Y_train)

print(grid_result.best_score, grid_result.best_params_)





RuntimeError: Cannot clone object <keras.wrappers.scikit_learn.KerasClassifier object at 0x000000C5D6E7E7B8>, as the constructor either does not set or modifies parameter layers

In [4]:
import tensorflow as tf
